In [1]:
from imblearn.over_sampling import ADASYN
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import os
from collections import Counter


In [2]:
# Chargement de la dataset
def load_images_and_labels(dataset_path, img_size=(224, 224)):
    images, labels = [], []
    for label_dir in os.listdir(dataset_path):
        for img_file in os.listdir(os.path.join(dataset_path, label_dir)):
            img_path = os.path.join(dataset_path, label_dir, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, img_size)
            images.append(image)
            labels.append(label_dir)
    return np.array(images), np.array(labels)


In [3]:
images, labels = load_images_and_labels("./wheat_leaf")
images = images / 255.0  # Normalisation des images



In [4]:
# Encodage des étiquettes
lb = LabelBinarizer()
labels = lb.fit_transform(labels)  # Si 'labels' sont déjà sous forme d'indices, sinon vous pouvez utiliser 'to_categorical' pour les encoder.



In [5]:
# Division en ensembles train, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
# Vérification de la forme des étiquettes
print("Forme de y_train :", y_train.shape)



Forme de y_train : (284, 3)


In [7]:
# Vérification de la distribution des classes dans le training set
class_distribution = Counter(np.argmax(y_train, axis=1))  # Si y_train est en one-hot, il faut prendre l'argmax pour avoir les classes
print("Distribution des classes dans le training set :", class_distribution)



Distribution des classes dans le training set : Counter({2: 150, 1: 70, 0: 64})


In [8]:
# Application d'ADASYN pour équilibrer les classes
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Aplatir les données pour ADASYN
adasyn = ADASYN(random_state=42, n_neighbors=5)
X_augmented, y_augmented = adasyn.fit_resample(X_train_flattened, np.argmax(y_train, axis=1))  # Utilisation de l'argmax pour y_train



In [9]:
# Reshape des données augmentées
X_augmented = X_augmented.reshape(-1, 224, 224, 3)



In [10]:
from keras.utils import to_categorical

# Convertir y_augmented en one-hot encoding pour correspondre à y_train
y_augmented_one_hot = to_categorical(y_augmented, num_classes=3)

# Fusionner les étiquettes (y)
y_augmented_train = np.concatenate((y_train, y_augmented_one_hot))

# Fusionner les données (X)
X_augmented_train = np.vstack((X_train, X_augmented))

print("Shape of X_augmented_train:", X_augmented_train.shape)
print("Shape of y_augmented_train:", y_augmented_train.shape)

# Vérification après fusion
print("Taille de l'ensemble entraînement augmenté :", X_augmented_train.shape, y_augmented_train.shape)



Shape of X_augmented_train: (730, 224, 224, 3)
Shape of y_augmented_train: (730, 3)
Taille de l'ensemble entraînement augmenté : (730, 224, 224, 3) (730, 3)


### ___________________________________ Xception _______________________________________________

In [11]:
from tensorflow.keras.applications import Xception

# Construction du modèle MobileNetV2
Xception_model = Xception(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
Xception_model.trainable = False



In [12]:
model = models.Sequential([
    Xception_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])



In [13]:
# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])



In [14]:
# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)



X_augmented_train dtype: float64
y_augmented_train dtype: float64


In [15]:
# Si les étiquettes ne sont pas déjà one-hot, appliquez `to_categorical`
if len(y_train.shape) == 1:
    y_train_one_hot = to_categorical(y_train, num_classes=3)
else:
    y_train_one_hot = y_train  # Les étiquettes sont déjà en one-hot encoding

if len(y_valid.shape) == 1:
    y_valid_one_hot = to_categorical(y_valid, num_classes=3)
else:
    y_valid_one_hot = y_valid  # Les étiquettes sont déjà en one-hot encoding

if len(y_augmented_train.shape) == 1:
    y_augmented_train_one_hot = to_categorical(y_augmented_train, num_classes=3)
else:
    y_augmented_train_one_hot = y_augmented_train  # Les étiquettes sont déjà en one-hot encoding

# Vérification des formes des étiquettes avant de continuer
print("Shape of y_train_one_hot:", y_train_one_hot.shape)
print("Shape of y_valid_one_hot:", y_valid_one_hot.shape)
print("Shape of y_augmented_train_one_hot:", y_augmented_train_one_hot.shape)

# Entraînement du modèle avec les données préparées
model.fit(X_augmented_train, y_augmented_train_one_hot, epochs=20, validation_data=(X_valid, y_valid_one_hot))


Shape of y_train_one_hot: (284, 3)
Shape of y_valid_one_hot: (61, 3)
Shape of y_augmented_train_one_hot: (730, 3)
Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - accuracy: 0.6884 - loss: 0.7104 - val_accuracy: 0.8361 - val_loss: 0.3656
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.9488 - loss: 0.1522 - val_accuracy: 0.8525 - val_loss: 0.3586
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 70s 3s/step - accuracy: 0.9902 - loss: 0.0796 - val_accuracy: 0.9180 - val_loss: 0.2566
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.9981 - loss: 0.0334 - val_accuracy: 0.9016 - val_loss: 0.2737
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 1.0000 - loss: 0.0247 - val_accuracy: 0.9344 - val_loss: 0.2435
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 1.0000 - loss: 0.0190 - val_accuracy: 0.8852 - val_loss: 0.3127
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 1.0000 - loss: 0.0139 - val_accuracy: 0.9180 - val_loss: 0.2492
Epo

In [16]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")



2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.9577 - loss: 0.1340
Précision sur l'ensemble test : 0.97


In [17]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)



2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 8s/step


In [18]:
from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")

# Si lb.classes_ contient des entiers, convertissez-les en chaînes
target_names = [str(cls) for cls in lb.classes_]

# Maintenant, passez-les à la fonction classification_report
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))



Matrice de confusion :
[[17  0  1]
 [ 0 13  0]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.94      0.94      0.94        18
    septoria       1.00      1.00      1.00        13
 stripe_rust       0.97      0.97      0.97        31

    accuracy                           0.97        62
   macro avg       0.97      0.97      0.97        62
weighted avg       0.97      0.97      0.97        62



### _____________________________________ DenceNet121 _______________________________________

In [11]:
from tensorflow.keras.applications import DenseNet121

# Construction du modèle MobileNetV2
DenseNet121_model = DenseNet121(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
DenseNet121_model.trainable = False

model = models.Sequential([
    DenseNet121_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


# Si les étiquettes ne sont pas déjà one-hot, appliquez `to_categorical`
if len(y_train.shape) == 1:
    y_train_one_hot = to_categorical(y_train, num_classes=3)
else:
    y_train_one_hot = y_train  # Les étiquettes sont déjà en one-hot encoding

if len(y_valid.shape) == 1:
    y_valid_one_hot = to_categorical(y_valid, num_classes=3)
else:
    y_valid_one_hot = y_valid  # Les étiquettes sont déjà en one-hot encoding

if len(y_augmented_train.shape) == 1:
    y_augmented_train_one_hot = to_categorical(y_augmented_train, num_classes=3)
else:
    y_augmented_train_one_hot = y_augmented_train  # Les étiquettes sont déjà en one-hot encoding

# Vérification des formes des étiquettes avant de continuer
print("Shape of y_train_one_hot:", y_train_one_hot.shape)
print("Shape of y_valid_one_hot:", y_valid_one_hot.shape)
print("Shape of y_augmented_train_one_hot:", y_augmented_train_one_hot.shape)




X_augmented_train dtype: float64
y_augmented_train dtype: float64
Shape of y_train_one_hot: (284, 3)
Shape of y_valid_one_hot: (61, 3)
Shape of y_augmented_train_one_hot: (730, 3)


In [12]:
# Entraînement du modèle avec les données préparées
model.fit(X_augmented_train, y_augmented_train_one_hot, epochs=20, validation_data=(X_valid, y_valid_one_hot))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 134s 4s/step - accuracy: 0.5519 - loss: 1.0293 - val_accuracy: 0.8852 - val_loss: 0.3094
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9522 - loss: 0.1818 - val_accuracy: 0.9344 - val_loss: 0.2050
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.9928 - loss: 0.0992 - val_accuracy: 0.9836 - val_loss: 0.1558
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.9976 - loss: 0.0499 - val_accuracy: 0.9836 - val_loss: 0.1256
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 1.0000 - loss: 0.0339 - val_accuracy: 0.9344 - val_loss: 0.1275
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 1.0000 - loss: 0.0252 - val_accuracy: 0.9836 - val_loss: 0.1015
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 1.0000 - loss: 0.0177 - val_accuracy: 0.9836 - val_loss: 0.0944
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 1.0000 - loss: 0.0138 - val_accuracy: 0.9836 - val_loss

In [13]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9785 - loss: 0.0422
Précision sur l'ensemble test : 0.97


In [14]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 16s/step


In [15]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")

# Si lb.classes_ contient des entiers, convertissez-les en chaînes
target_names = [str(cls) for cls in lb.classes_]

# Maintenant, passez-les à la fonction classification_report
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))

Matrice de confusion :
[[18  0  0]
 [ 0 11  2]
 [ 0  0 31]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00        18
    septoria       1.00      0.85      0.92        13
 stripe_rust       0.94      1.00      0.97        31

    accuracy                           0.97        62
   macro avg       0.98      0.95      0.96        62
weighted avg       0.97      0.97      0.97        62



### ___________________________________________ DenceNet169 ____________________________________________________

In [16]:
from tensorflow.keras.applications import DenseNet169

# Construction du modèle MobileNetV2
DenseNet169_model = DenseNet169(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
DenseNet169_model.trainable = False

model = models.Sequential([
    DenseNet169_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


# Si les étiquettes ne sont pas déjà one-hot, appliquez `to_categorical`
if len(y_train.shape) == 1:
    y_train_one_hot = to_categorical(y_train, num_classes=3)
else:
    y_train_one_hot = y_train  # Les étiquettes sont déjà en one-hot encoding

if len(y_valid.shape) == 1:
    y_valid_one_hot = to_categorical(y_valid, num_classes=3)
else:
    y_valid_one_hot = y_valid  # Les étiquettes sont déjà en one-hot encoding

if len(y_augmented_train.shape) == 1:
    y_augmented_train_one_hot = to_categorical(y_augmented_train, num_classes=3)
else:
    y_augmented_train_one_hot = y_augmented_train  # Les étiquettes sont déjà en one-hot encoding

# Vérification des formes des étiquettes avant de continuer
print("Shape of y_train_one_hot:", y_train_one_hot.shape)
print("Shape of y_valid_one_hot:", y_valid_one_hot.shape)
print("Shape of y_augmented_train_one_hot:", y_augmented_train_one_hot.shape)



X_augmented_train dtype: float64
y_augmented_train dtype: float64
Shape of y_train_one_hot: (284, 3)
Shape of y_valid_one_hot: (61, 3)
Shape of y_augmented_train_one_hot: (730, 3)


In [17]:
# Entraînement du modèle avec les données préparées
model.fit(X_augmented_train, y_augmented_train_one_hot, epochs=20, validation_data=(X_valid, y_valid_one_hot))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 188s 5s/step - accuracy: 0.7436 - loss: 0.5524 - val_accuracy: 0.8525 - val_loss: 0.3107
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.9761 - loss: 0.0945 - val_accuracy: 0.9672 - val_loss: 0.1698
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.9978 - loss: 0.0381 - val_accuracy: 0.9180 - val_loss: 0.2352
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 1.0000 - loss: 0.0200 - val_accuracy: 0.9508 - val_loss: 0.1729
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 1.0000 - loss: 0.0134 - val_accuracy: 0.9508 - val_loss: 0.1311
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 70s 3s/step - accuracy: 1.0000 - loss: 0.0103 - val_accuracy: 0.9344 - val_loss: 0.1966
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 1.0000 - loss: 0.0074 - val_accuracy: 0.9344 - val_loss: 0.1899
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 0.9344 - val_loss

In [18]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.9577 - loss: 0.0878
Précision sur l'ensemble test : 0.97


In [19]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 49s 23s/step


In [20]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")

# Si lb.classes_ contient des entiers, convertissez-les en chaînes
target_names = [str(cls) for cls in lb.classes_]

# Maintenant, passez-les à la fonction classification_report
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))

Matrice de confusion :
[[16  0  2]
 [ 0 13  0]
 [ 0  0 31]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      0.89      0.94        18
    septoria       1.00      1.00      1.00        13
 stripe_rust       0.94      1.00      0.97        31

    accuracy                           0.97        62
   macro avg       0.98      0.96      0.97        62
weighted avg       0.97      0.97      0.97        62



### ______________________________________________ ResNet50V2 ______________________________________________

In [21]:
from tensorflow.keras.applications import ResNet50V2

# Construction du modèle MobileNetV2
ResNet50V2_model = ResNet50V2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
ResNet50V2_model.trainable = False

model = models.Sequential([
    ResNet50V2_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


# Si les étiquettes ne sont pas déjà one-hot, appliquez `to_categorical`
if len(y_train.shape) == 1:
    y_train_one_hot = to_categorical(y_train, num_classes=3)
else:
    y_train_one_hot = y_train  # Les étiquettes sont déjà en one-hot encoding

if len(y_valid.shape) == 1:
    y_valid_one_hot = to_categorical(y_valid, num_classes=3)
else:
    y_valid_one_hot = y_valid  # Les étiquettes sont déjà en one-hot encoding

if len(y_augmented_train.shape) == 1:
    y_augmented_train_one_hot = to_categorical(y_augmented_train, num_classes=3)
else:
    y_augmented_train_one_hot = y_augmented_train  # Les étiquettes sont déjà en one-hot encoding

# Vérification des formes des étiquettes avant de continuer
print("Shape of y_train_one_hot:", y_train_one_hot.shape)
print("Shape of y_valid_one_hot:", y_valid_one_hot.shape)
print("Shape of y_augmented_train_one_hot:", y_augmented_train_one_hot.shape)



X_augmented_train dtype: float64
y_augmented_train dtype: float64
Shape of y_train_one_hot: (284, 3)
Shape of y_valid_one_hot: (61, 3)
Shape of y_augmented_train_one_hot: (730, 3)


In [22]:
# Entraînement du modèle avec les données préparées
model.fit(X_augmented_train, y_augmented_train_one_hot, epochs=20, validation_data=(X_valid, y_valid_one_hot))



Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step - accuracy: 0.6090 - loss: 1.0597 - val_accuracy: 0.8689 - val_loss: 0.2242
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9965 - loss: 0.0722 - val_accuracy: 0.8689 - val_loss: 0.2699
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 1.0000 - loss: 0.0205 - val_accuracy: 0.8852 - val_loss: 0.2084
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 1.0000 - loss: 0.0101 - val_accuracy: 0.8689 - val_loss: 0.2390
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 0.8689 - val_loss: 0.2667
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 0.8689 - val_loss: 0.2217
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0.8689 - val_loss: 0.2434
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.8689 - val_loss:

In [23]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")



2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9573 - loss: 0.1306
Précision sur l'ensemble test : 0.95


In [24]:

# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)



1/2 ━━━━━━━━━━━━━━━━━━━━ 9s 10s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002F6C2F532E0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 10s/step


In [25]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")

# Si lb.classes_ contient des entiers, convertissez-les en chaînes
target_names = [str(cls) for cls in lb.classes_]

# Maintenant, passez-les à la fonction classification_report
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))



Matrice de confusion :
[[17  0  1]
 [ 0 12  1]
 [ 0  1 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      0.94      0.97        18
    septoria       0.92      0.92      0.92        13
 stripe_rust       0.94      0.97      0.95        31

    accuracy                           0.95        62
   macro avg       0.95      0.95      0.95        62
weighted avg       0.95      0.95      0.95        62



### _____________________________________________________ MobileNetV2 __________________________________________--

In [26]:
from tensorflow.keras.applications import MobileNetV2

# Construction du modèle MobileNetV2
MobileNetV2_model = MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
MobileNetV2_model.trainable = False

model = models.Sequential([
    MobileNetV2_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


# Si les étiquettes ne sont pas déjà one-hot, appliquez `to_categorical`
if len(y_train.shape) == 1:
    y_train_one_hot = to_categorical(y_train, num_classes=3)
else:
    y_train_one_hot = y_train  # Les étiquettes sont déjà en one-hot encoding

if len(y_valid.shape) == 1:
    y_valid_one_hot = to_categorical(y_valid, num_classes=3)
else:
    y_valid_one_hot = y_valid  # Les étiquettes sont déjà en one-hot encoding

if len(y_augmented_train.shape) == 1:
    y_augmented_train_one_hot = to_categorical(y_augmented_train, num_classes=3)
else:
    y_augmented_train_one_hot = y_augmented_train  # Les étiquettes sont déjà en one-hot encoding

# Vérification des formes des étiquettes avant de continuer
print("Shape of y_train_one_hot:", y_train_one_hot.shape)
print("Shape of y_valid_one_hot:", y_valid_one_hot.shape)
print("Shape of y_augmented_train_one_hot:", y_augmented_train_one_hot.shape)



X_augmented_train dtype: float64
y_augmented_train dtype: float64
Shape of y_train_one_hot: (284, 3)
Shape of y_valid_one_hot: (61, 3)
Shape of y_augmented_train_one_hot: (730, 3)


In [27]:
# Entraînement du modèle avec les données préparées
model.fit(X_augmented_train, y_augmented_train_one_hot, epochs=20, validation_data=(X_valid, y_valid_one_hot))



Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 979ms/step - accuracy: 0.6412 - loss: 0.7339 - val_accuracy: 0.8689 - val_loss: 0.2819
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 617ms/step - accuracy: 0.9582 - loss: 0.1246 - val_accuracy: 0.9672 - val_loss: 0.1668
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 539ms/step - accuracy: 0.9873 - loss: 0.0501 - val_accuracy: 0.9180 - val_loss: 0.1660
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 495ms/step - accuracy: 1.0000 - loss: 0.0149 - val_accuracy: 0.9180 - val_loss: 0.1599
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 492ms/step - accuracy: 1.0000 - loss: 0.0105 - val_accuracy: 0.9180 - val_loss: 0.1705
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 537ms/step - accuracy: 1.0000 - loss: 0.0075 - val_accuracy: 0.9180 - val_loss: 0.1683
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 18s 787ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 0.9180 - val_loss: 0.1634
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 21s 813ms/step - accuracy: 1.0000 - loss: 0.0037 - val_accu

In [28]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")




2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9150 - loss: 0.1541
Précision sur l'ensemble test : 0.92


In [29]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)



2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step


In [30]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")

# Si lb.classes_ contient des entiers, convertissez-les en chaînes
target_names = [str(cls) for cls in lb.classes_]

# Maintenant, passez-les à la fonction classification_report
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))



Matrice de confusion :
[[15  0  3]
 [ 0 12  1]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.94      0.83      0.88        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.88      0.97      0.92        31

    accuracy                           0.92        62
   macro avg       0.94      0.91      0.92        62
weighted avg       0.92      0.92      0.92        62

